In [ ]:
class AMMTokenomicsCalculator:
    """Precise AMM math for tokenomics design"""
    
    def __init__(self, 
                 usdt_pool: float,
                 mind_pool: float,
                 fee_rate: float = 0.003):
        self.x = usdt_pool
        self.y = mind_pool
        self.k = usdt_pool * mind_pool
        self.fee = fee_rate
        self.current_price = usdt_pool / mind_pool
    
    def price_after_trades(self, buyback_usd: float, emission_tokens: float) -> dict:
        """
        Calculate exact price after buyback and emission selling
        
        Math:
        1. Buyback: x₁ = x + b(1-f), y₁ = k/x₁
        2. Sell: y₂ = y₁ + e(1-f), x₂ = k/y₂
        3. Final price: p_new = x₂/y₂
        """
        # Step 1: Buyback (buy MIND with USDT)
        x1 = self.x + buyback_usd * (1 - self.fee)
        y1 = self.k / x1
        
        # Step 2: Emission selling (sell MIND for USDT)
        y2 = y1 + emission_tokens * (1 - self.fee)
        x2 = self.k / y2
        
        # Final price
        p_new = x2 / y2
        
        # Calculate percentage change
        price_change = (p_new / self.current_price - 1)
        
        return {
            'initial_price': self.current_price,
            'final_price': p_new,
            'price_change_pct': price_change * 100,
            'x_final': x2,
            'y_final': y2,
            'mind_bought': self.y - y1,
            'usdt_from_selling': x1 - x2
        }
    
    def find_buyback_for_target_appreciation(self,
                                             emission_tokens: float,
                                             target_annual_appreciation: float,
                                             max_iterations: int = 100,
                                             tolerance: float = 1e-6) -> dict:
        """
        Binary search to find exact buyback needed for target appreciation
        
        We want: p_new = p * (1 + r/365)^(1/365) for daily compounding
        
        Where p_new = x₂/y₂ after both buyback and selling
        """
        # Use compound interest formula for daily compounding
        target_daily_appreciation = (1 + target_annual_appreciation) ** (1/365) - 1
        target_price = self.current_price * (1 + target_daily_appreciation)
        
        # Binary search bounds
        low = 0.0
        high = emission_tokens * self.current_price * 10  # Upper bound
    
        for iteration in range(max_iterations):
            mid = (low + high) / 2
            result = self.price_after_trades(mid, emission_tokens)
            
            if abs(result['final_price'] - target_price) < tolerance:
                return {
                    'required_buyback_usd': mid,
                    'target_price': target_price,
                    'achieved_price': result['final_price'],
                    'target_daily_appreciation_pct': target_daily_appreciation * 100,
                    'achieved_daily_appreciation_pct': result['price_change_pct'],
                    'annual_appreciation_compound': ((result['final_price'] / self.current_price) ** 365 - 1) * 100,
                    'emission_tokens': emission_tokens,
                    'iterations': iteration + 1
                }
            
            if result['final_price'] < target_price:
                low = mid  # Need more buyback
            else:
                high = mid  # Too much buyback
        
        # Return best approximation
        result = self.price_after_trades(mid, emission_tokens)
        return {
            'required_buyback_usd': mid,
            'target_price': target_price,
            'achieved_price': result['final_price'],
            'target_daily_appreciation_pct': target_daily_appreciation * 100,
            'achieved_daily_appreciation_pct': result['price_change_pct'],
            'annual_appreciation_compound': ((result['final_price'] / self.current_price) ** 365 - 1) * 100,
            'emission_tokens': emission_tokens,
            'iterations': max_iterations,
            'converged': False
        }
    
    def find_max_sustainable_emission(
        self,
        daily_revenue_usd: float,
        target_annual_appreciation: float,
        miner_sell_ratio: float = 0.5,
        max_iterations: int = 100,
        tolerance: float = 1.0
    ) -> dict:
        """
        Binary search to find max emission that maintains target appreciation
        
        Given: daily_revenue (buyback capacity)
        Find: max daily_emission where price still appreciates at target rate
        """
        target_daily_appreciation = target_annual_appreciation / 365
        target_price = self.current_price * (1 + target_daily_appreciation)
        
        # Binary search bounds
        low = 0.0
        high = daily_revenue_usd / (self.current_price * miner_sell_ratio) * 10
        
        best_emission = 0.0
        
        for iteration in range(max_iterations):
            mid = (low + high) / 2
            emission_sold = mid * miner_sell_ratio
            
            result = self.price_after_trades(daily_revenue_usd, emission_sold)
            
            if abs(result['final_price'] - target_price) < tolerance * self.current_price * 1e-4:
                return {
                    'max_sustainable_emission': mid,
                    'daily_revenue': daily_revenue_usd,
                    'emission_sold': emission_sold,
                    'target_price': target_price,
                    'achieved_price': result['final_price'],
                    'annual_inflation_pct': (mid * 365 / (self.y * (1 + self.x/self.current_price))) * 100,
                    'miner_sell_ratio': miner_sell_ratio,
                    'iterations': iteration + 1
                }
            
            if result['final_price'] < target_price:
                high = mid  # Emission too high, price dropped
                best_emission = mid
            else:
                low = mid  # Can support more emission
        
        # Return best approximation
        emission_sold = best_emission * miner_sell_ratio
        result = self.price_after_trades(daily_revenue_usd, emission_sold)
        
        return {
            'max_sustainable_emission': best_emission,
            'daily_revenue': daily_revenue_usd,
            'emission_sold': emission_sold,
            'target_price': target_price,
            'achieved_price': result['final_price'],
            'annual_inflation_pct': (best_emission * 365 / (self.y * (1 + self.x/self.current_price))) * 100,
            'miner_sell_ratio': miner_sell_ratio,
            'iterations': max_iterations,
            'converged': False
        }

# Initialize with your parameters
amm = AMMTokenomicsCalculator(
    usdt_pool=300_000,  # Initial USDT
    mind_pool=300_000,  # Initial MIND (starting price = $1.00)
    fee_rate=0.003
)

print("="*70)
print("PRECISE AMM CALCULATIONS")
print(f"Initial Pool: {amm.x:,.0f} USDT / {amm.y:,.0f} MIND")
print(f"Initial Price: ${amm.current_price:.4f}")
print(f"Pool Constant k: {amm.k:,.0f}")
print("="*70)

# Test 1: What buyback needed for 10% annual appreciation with 22,245 emission?
print("\n📊 TEST 1: Bitcoin-style emission (22,245/day)")
print("Target: 10% annual appreciation")

result1 = amm.find_buyback_for_target_appreciation(
    emission_tokens=22_245 * 0.5,  # Assume 50% sold
    target_annual_appreciation=0.10
)

print(f"\nDaily Emission: 22,245 MIND")
print(f"Miner Sell Ratio: 50%")
print(f"Tokens Sold: {22_245 * 0.5:,.0f} MIND")
print(f"\nRequired Daily Buyback: ${result1['required_buyback_usd']:,.2f}")
print(f"Target Daily Appreciation: {result1['target_daily_appreciation_pct']:.6f}%")
print(f"Achieved Appreciation: {result1['achieved_daily_appreciation_pct']:.6f}%")
print(f"Converged in {result1['iterations']} iterations")

# Test 2: Given $5k/day revenue, what's max sustainable emission?
print("\n" + "="*70)
print("📊 TEST 2: Revenue-Constrained Emission")
print("Given: $5,000/day revenue")
print("Target: 10% annual appreciation")

result2 = amm.find_max_sustainable_emission(
    daily_revenue_usd=5_000,
    target_annual_appreciation=0.10,
    miner_sell_ratio=0.5
)

print(f"\nMax Sustainable Emission: {result2['max_sustainable_emission']:,.0f} MIND/day")
print(f"Tokens Sold (50%): {result2['emission_sold']:,.0f} MIND")
print(f"Annual Inflation Rate: {result2['annual_inflation_pct']:.2f}%")
print(f"Target Price: ${result2['target_price']:.4f}")
print(f"Achieved Price: ${result2['achieved_price']:.4f}")

# Test 3: Different revenue scenarios
print("\n" + "="*70)
print("📊 TEST 3: Revenue Scenario Analysis")
print("="*70)

scenarios = [
    ("Conservative", 2_000),
    ("Moderate", 5_000),
    ("Aggressive", 10_000),
    ("Very Aggressive", 20_000),
]

for name, revenue in scenarios:
    result = amm.find_max_sustainable_emission(
        daily_revenue_usd=revenue,
        target_annual_appreciation=0.10,
        miner_sell_ratio=0.5
    )
    
    print(f"\n{name}: ${revenue:,}/day revenue")
    print(f"  Max Emission: {result['max_sustainable_emission']:,.0f} MIND/day")
    print(f"  Annual Inflation: {result['annual_inflation_pct']:.2f}%")
    print(f"  New Price: ${result['achieved_price']:.4f}")

# Test 4: Show the math step-by-step for one example
print("\n" + "="*70)
print("📊 TEST 4: Step-by-Step AMM Math")
print("="*70)
print("\nScenario: $5k buyback, 10k MIND emission sold")

detailed = amm.price_after_trades(
    buyback_usd=5_000,
    emission_tokens=10_000
)

print(f"\nInitial State:")
print(f"  x (USDT pool): {amm.x:,.0f}")
print(f"  y (MIND pool): {amm.y:,.0f}")
print(f"  k = x * y: {amm.k:,.0f}")
print(f"  Price: ${detailed['initial_price']:.4f}")

print(f"\nStep 1: Buyback (buy MIND with $5,000 USDT)")
x1 = amm.x + 5_000 * (1 - amm.fee)
y1 = amm.k / x1
print(f"  x₁ = {amm.x:,.0f} + 5000*(1-0.003) = {x1:,.0f}")
print(f"  y₁ = {amm.k:,.0f} / {x1:,.0f} = {y1:,.0f}")
print(f"  MIND bought: {detailed['mind_bought']:,.2f}")
print(f"  Price after buyback: ${x1/y1:.4f}")

print(f"\nStep 2: Emission Selling (sell 10,000 MIND)")
y2 = y1 + 10_000 * (1 - amm.fee)
x2 = amm.k / y2
print(f"  y₂ = {y1:,.0f} + 10000*(1-0.003) = {y2:,.0f}")
print(f"  x₂ = {amm.k:,.0f} / {y2:,.0f} = {x2:,.0f}")
print(f"  USDT received: {detailed['usdt_from_selling']:,.2f}")

print(f"\nFinal State:")
print(f"  x (USDT pool): {detailed['x_final']:,.0f}")
print(f"  y (MIND pool): {detailed['y_final']:,.0f}")
print(f"  Final Price: ${detailed['final_price']:.4f}")
print(f"  Price Change: {detailed['price_change_pct']:+.4f}%")

In [ ]:
# ============================================================================
# PHASE-BY-PHASE ANALYSIS WITH REAL REVENUE DATA
# ============================================================================

# User-configurable parameters at the top
TARGET_ANNUAL_APPRECIATION = 0.10  # 10% per year
MINER_SELL_RATIO = 0.5  # 50% of emissions sold by miners

# Initial pool state
initial_usdt_pool = 300_000
initial_mind_pool = 300_000
initial_price = initial_usdt_pool / initial_mind_pool
fee_rate = 0.003

# Phase data from your revenue projections
phases = [
    {
        'month': 1,
        'phase': 'Testnet Pilot',
        'dau': 100,
        'prompts_per_user': 80,
        'fee_per_inference': 0.10,
        'monthly_revenue': 800
    },
    {
        'month': 2,
        'phase': 'Testnet Beta',
        'dau': 250,
        'prompts_per_user': 80,
        'fee_per_inference': 0.10,
        'monthly_revenue': 2_000
    },
    {
        'month': 3,
        'phase': 'Ecosystem Growth',
        'dau': 600,
        'prompts_per_user': 90,
        'fee_per_inference': 0.10,
        'monthly_revenue': 5_400
    },
    {
        'month': 4,
        'phase': 'Pre-TGE Warmup',
        'dau': 1_200,
        'prompts_per_user': 90,
        'fee_per_inference': 0.10,
        'monthly_revenue': 10_800
    },
    {
        'month': 5,
        'phase': 'Mainnet / TGE',
        'dau': 2_100,
        'prompts_per_user': 100,
        'fee_per_inference': 0.10,
        'monthly_revenue': 21_000
    },
    {
        'month': 6,
        'phase': 'Post-Launch',
        'dau': 3_000,
        'prompts_per_user': 100,
        'fee_per_inference': 0.10,
        'monthly_revenue': 30_000
    },
    {
        'month': 7,
        'phase': 'Agent Infrastructure',
        'dau': 4_200,
        'prompts_per_user': 110,
        'fee_per_inference': 0.12,
        'monthly_revenue': 55_440
    },
    {
        'month': 8,
        'phase': 'Model Expansion',
        'dau': 6_000,
        'prompts_per_user': 110,
        'fee_per_inference': 0.12,
        'monthly_revenue': 79_200
    },
    {
        'month': 9,
        'phase': 'Launchpad Live',
        'dau': 8_500,
        'prompts_per_user': 120,
        'fee_per_inference': 0.12,
        'monthly_revenue': 122_400
    },
    {
        'month': 10,
        'phase': 'Specialized Clusters',
        'dau': 11_000,
        'prompts_per_user': 120,
        'fee_per_inference': 0.15,
        'monthly_revenue': 198_000
    },
    {
        'month': 11,
        'phase': 'Scaling Mainstream',
        'dau': 14_000,
        'prompts_per_user': 130,
        'fee_per_inference': 0.15,
        'monthly_revenue': 273_000
    },
    {
        'month': 12,
        'phase': 'Protocol Maturity',
        'dau': 18_000,
        'prompts_per_user': 130,
        'fee_per_inference': 0.15,
        'monthly_revenue': 351_000
    }
]

print("="*80)
print("PHASE-BY-PHASE TOKENOMICS ANALYSIS")
print(f"Target Annual Appreciation: {TARGET_ANNUAL_APPRECIATION*100:.0f}%")
print(f"Miner Sell Ratio: {MINER_SELL_RATIO*100:.0f}%")
print("="*80)


for phase in phases:
    # Calculate daily revenue (monthly / 30 days)
    daily_revenue = phase['monthly_revenue'] / 30
    
    # Initialize AMM for this phase
    amm = AMMTokenomicsCalculator(
        usdt_pool=initial_usdt_pool,
        mind_pool=initial_mind_pool,
        fee_rate=fee_rate
    )
    
    # Find max sustainable emission for this revenue level
    result = amm.find_max_sustainable_emission(
        daily_revenue_usd=daily_revenue,
        target_annual_appreciation=TARGET_ANNUAL_APPRECIATION,
        miner_sell_ratio=MINER_SELL_RATIO
    )
    
    print(f"\n{'='*80}")
    print(f"MONTH {phase['month']}: {phase['phase']}")
    print(f"{'='*80}")
    
    print(f"\n📊 Business Metrics:")
    print(f"   Daily Active Users: {phase['dau']:,}")
    print(f"   Prompts/User/Month: {phase['prompts_per_user']}")
    print(f"   Fee per Inference: ${phase['fee_per_inference']:.2f}")
    print(f"   Monthly Revenue: ${phase['monthly_revenue']:,}")
    print(f"   Daily Revenue: ${daily_revenue:,.2f}")
    
    print(f"\n💰 Tokenomics Results:")
    print(f"   Max Sustainable Emission: {result['max_sustainable_emission']:,.0f} MIND/day")
    print(f"   Tokens Sold by Miners: {result['emission_sold']:,.0f} MIND/day")
    print(f"   Tokens Held by Miners: {result['max_sustainable_emission'] - result['emission_sold']:,.0f} MIND/day")
    print(f"   Annual Inflation Rate: {result['annual_inflation_pct']:.2f}%")
    
    # Calculate monthly emission
    monthly_emission = result['max_sustainable_emission'] * 30
    annual_emission = result['max_sustainable_emission'] * 365
    
    print(f"\n📅 Emission Rates:")
    print(f"   Monthly Emission: {monthly_emission:,.0f} MIND")
    print(f"   Annual Emission: {annual_emission:,.0f} MIND")
    
    # Calculate how many miners this can support
    # Assume each miner needs ~$100/day in rewards to be profitable
    daily_reward_pool_usd = result['max_sustainable_emission'] * amm.current_price
    estimated_miners = daily_reward_pool_usd / 100
    
    print(f"\n👷 Miner Economics:")
    print(f"   Daily Reward Pool: ${daily_reward_pool_usd:,.2f}")
    print(f"   Est. Miners Supported (~$100/day each): {estimated_miners:.0f}")
    print(f"   Reward per Miner: {result['max_sustainable_emission']/max(1, estimated_miners):,.0f} MIND/day")
    
    print(f"\n✅ Price Stability Check:")
    print(f"   Current Price: ${amm.current_price:.4f}")
    print(f"   Target Price (next day): ${result['target_price']:.4f}")
    print(f"   Achieved Price: ${result['achieved_price']:.4f}")
    
    # Calculate if this is net positive or negative
    buyback_tokens = daily_revenue / amm.current_price
    sell_pressure_tokens = result['emission_sold']
    net_flow = buyback_tokens - sell_pressure_tokens
    
    print(f"\n🔄 Daily Token Flows:")
    print(f"   Buyback Pressure: {buyback_tokens:,.0f} MIND (${daily_revenue:,.0f})")
    print(f"   Sell Pressure: {sell_pressure_tokens:,.0f} MIND")
    print(f"   Net Flow: {net_flow:+,.0f} MIND {'✅' if net_flow > 0 else '⚠️'}")
    
    if net_flow > 0:
        ratio = buyback_tokens / sell_pressure_tokens
        print(f"   Buyback/Sell Ratio: {ratio:.2f}x (Net Positive)")
    else:
        print(f"   ⚠️  WARNING: Sell pressure exceeds buyback capacity")

# Summary comparison
print(f"\n{'='*80}")
print("SUMMARY: EMISSION SCHEDULE ACROSS PHASES")
print(f"{'='*80}\n")

summary_data = []
for phase in phases:
    daily_revenue = phase['monthly_revenue'] / 30
    amm = AMMTokenomicsCalculator(initial_usdt_pool, initial_mind_pool, fee_rate)
    result = amm.find_max_sustainable_emission(
        daily_revenue_usd=daily_revenue,
        target_annual_appreciation=TARGET_ANNUAL_APPRECIATION,
        miner_sell_ratio=MINER_SELL_RATIO
    )
    
    summary_data.append({
        'Month': phase['month'],
        'Phase': phase['phase'],
        'Revenue/Day': f"${daily_revenue:,.0f}",
        'Emission/Day': f"{result['max_sustainable_emission']:,.0f}",
        'Inflation %': f"{result['annual_inflation_pct']:.2f}%",
        'Status': '✅' if result['max_sustainable_emission'] > 0 else '⚠️'
    })

# Print as table
import pandas as pd
summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print(f"\n{'='*80}")
print("KEY INSIGHTS:")
print(f"{'='*80}")
print(f"\n1. Early Phases (Months 1-4):")
print(f"   - Low revenue means very limited sustainable emission")
print(f"   - Consider lower initial emissions or subsidize from treasury")
print(f"   - Focus on testnet rewards rather than mainnet mining")

print(f"\n2. TGE Phase (Month 5):")
print(f"   - Revenue crosses critical threshold for sustainable mining")
print(f"   - This is the optimal time for mainnet launch")

print(f"\n3. Growth Phase (Months 6-12):")
print(f"   - Rapidly increasing emission capacity")
print(f"   - Can support growing miner base")
print(f"   - Maintain {TARGET_ANNUAL_APPRECIATION*100:.0f}% price appreciation target")

print(f"\n4. Recommendation:")
print(f"   - Start with conservative emission in Month 5")
print(f"   - Gradually increase as revenue grows")
print(f"   - Always maintain buyback > sell pressure for price stability")